In [1]:
import pandas as pd
import json
import re

In [2]:
pos_file = '/Users/shiwei/Desktop/twitter_samples/positive_tweets.json'
neg_file = '/Users/shiwei/Desktop/twitter_samples/negative_tweets.json'

In [3]:
list1 = []
with open(pos_file, "r") as f:
    content = f.readlines() 
    for line in content:    
        pos_data = json.loads(line)
        list1.append(pos_data)
df_pos = pd.DataFrame(list1)

list2 = []
with open(neg_file, "r") as f:
    content = f.readlines() 
    for line in content:    
        neg_data = json.loads(line)
        list2.append(neg_data)
df_neg = pd.DataFrame(list2)

In [4]:
### mapping a dictionary of apostrophe words

appos = {
"aren't" : "are not",
"can't" : "cannot",
"cant" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"im" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"gg" : "going"
}

In [5]:
def preprocess_text(text):
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', text)
    text = re.sub('@[^\s]+','', text)
    text = text.lower().split()
    reformed = [appos[word] if word in appos else word for word in text]
    reformed = " ".join(reformed) 
    text = re.sub('&[^\s]+;', '', reformed)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +',' ', text)
    #text = re.sub(' [\w] ', ' ', text)
    return text.strip()

In [6]:
df_neg['clean_text'] = df_neg['text'].apply(lambda x: preprocess_text(x))
df_pos['clean_text'] = df_pos['text'].apply(lambda x: preprocess_text(x))

In [7]:
df_pos2 = pd.DataFrame(df_pos['clean_text'])
df_neg2 = pd.DataFrame(df_neg['clean_text'])

In [8]:
df_pos2.insert(0, 'label', 1)
df_neg2.insert(0, 'label', -1)

In [9]:
data = df_pos2.append(df_neg2, ignore_index=True)
data

/var/folders/g6/dzv0rfqn11j018hcgf4ky0080000gn/T/ipykernel_1815/3077277819.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = df_pos2.append(df_neg2, ignore_index=True)


,label,clean_text
0,1,followfriday for being top engaged members in ...
1,1,hey james how odd please call our contact cent...
2,1,we had a listen last night as you bleed is an ...
3,1,congrats
4,1,yeaaaah yippppy my accnt verified rqst has suc...
...,...,...
9995,-1,i wanna change my avi but usanele
9996,-1,my puppy broke her foot
9997,-1,where is all the jaebum baby pictures
9998,-1,but but mr ahmad maslan cooks too


In [85]:
data.to_excel('ceshi.xlsx')

# 下面开始进行sklearn各种分类器的机器学习

In [11]:
#Import feature engineering modules and test_train_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

#Import classification algorithm
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
#from xgboost import XGBClassifier

#Import modules to calculate accuracy and confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

## 下面使用TF-IDF方法将文本转换成向量，应用朴素贝叶斯分类器进行分类，给出分类效果评价

In [19]:
#TfidfVectorizer()函数是用TF-IDF算法将文本转换成数字向量，括号里设置必要的参数
#fit_transform()函数是正式传入文本，进行转换
tv = TfidfVectorizer(ngram_range=(1,3), stop_words='english') 
#X = tv.fit_transform(data['clean_text'])
X = tv.fit_transform(data['clean_text'])

# train_test_split函数可以随机切分训练集和测试集
# Xtrain：训练集数据；ytrain：训练集标签；Xtest：测试集数据；ytest：测试集标签
Xtrain, Xtest, ytrain, ytest = train_test_split(X, data['label'],
                                               test_size = 0.3, shuffle=True)

#开始训练
nb = MultinomialNB(alpha=6.5, fit_prior=False)
nb.fit(Xtrain,ytrain)
pred = nb.predict(Xtest)

#给出训练效果评分
print(accuracy_score(ytest,pred))
print(confusion_matrix(ytest,pred))
print(classification_report(ytest,pred))

0.7343333333333333
[[1119  348]
 [ 449 1084]]
              precision    recall  f1-score   support

          -1       0.71      0.76      0.74      1467
           1       0.76      0.71      0.73      1533

    accuracy                           0.73      3000
   macro avg       0.74      0.73      0.73      3000
weighted avg       0.74      0.73      0.73      3000



Naive Bayes with Grid Search Hyperparameter Tuning & 10-Fold Cross Validation - achieving higher accuracy over the mdoel without hyperparameter tuning
grid search：超参数选择，用于超参数优化，通过优化超参数之间的最优组合来改善模型性能。
grid search使用暴风搜索法，首先为不同的超参数设定一个值列表，然后计算机会遍历每个超参数的组合进行性能评估，选出性能最佳的参数组合。

In [20]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])

# 设置超参数组合
tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]
}

In [21]:
x_train, x_test, y_train, y_test = train_test_split(data['clean_text'], data['label'],
                                               test_size = 0.3, shuffle=True)

In [22]:
clf = GridSearchCV(text_clf, tuned_parameters, cv=10)
clf.fit(x_train, y_train)

print(classification_report(y_test, clf.predict(x_test), digits=4))
print(accuracy_score(y_test, clf.predict(x_test)))
print(confusion_matrix(y_test, clf.predict(x_test)))

              precision    recall  f1-score   support

          -1     0.7383    0.8209    0.7774      1485
           1     0.8028    0.7149    0.7563      1515

    accuracy                         0.7673      3000
   macro avg     0.7706    0.7679    0.7669      3000
weighted avg     0.7709    0.7673    0.7667      3000

0.7673333333333333
[[1219  266]
 [ 432 1083]]


## 下面使用线性SVM（LinearSVC）方法进行机器学习分类

with Grid Search Hyperparameter Tuning & 10-Fold Cross Validation
grid search：超参数选择，用于超参数优化，通过优化超参数之间的最优组合来改善模型性能。
grid search使用暴风搜索法，首先为不同的超参数设定一个值列表，然后计算机会遍历每个超参数的组合进行性能评估，选出性能最佳的参数组合。

In [23]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC())])

# 设置超参数组合
tuned_parameters = {
    'vect__ngram_range': [(1, 2), (1, 3), (1, 4)],
    'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__tol': [1, 1e-1, 1e-2, 1e-3]
}

In [24]:
x_train, x_test, y_train, y_test = train_test_split(data['clean_text'], data['label'],
                                               test_size = 0.3, shuffle=True)

In [25]:
clf = GridSearchCV(text_clf, tuned_parameters, cv=10) # 输入超参数
clf.fit(x_train, y_train) #开始训练

print(classification_report(y_test, clf.predict(x_test), digits=4))
print(accuracy_score(y_test, clf.predict(x_test)))
print(confusion_matrix(y_test, clf.predict(x_test)))

print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

              precision    recall  f1-score   support

          -1     0.7575    0.8095    0.7826      1501
           1     0.7951    0.7405    0.7668      1499

    accuracy                         0.7750      3000
   macro avg     0.7763    0.7750    0.7747      3000
weighted avg     0.7763    0.7750    0.7747      3000

0.775
[[1215  286]
 [ 389 1110]]
Best Score:  0.772
Best Params:  {'clf__tol': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 3)}


## 下面使用SVM（SVC）方法进行机器学习分类

In [26]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC())])

# 设置超参数组合
tuned_parameters = {
    'vect__ngram_range': [(1, 2), (1, 3), (1, 4)],
    'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    'clf__tol': [1, 1e-1, 1e-2, 1e-3]
}

In [27]:
x_train, x_test, y_train, y_test = train_test_split(data['clean_text'], data['label'],
                                               test_size = 0.3, shuffle=True)

In [28]:
clf = GridSearchCV(text_clf, tuned_parameters, cv=10) # 输入超参数
clf.fit(x_train, y_train) #开始训练

print(classification_report(y_test, clf.predict(x_test), digits=4))
print(accuracy_score(y_test, clf.predict(x_test)))
print(confusion_matrix(y_test, clf.predict(x_test)))

print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)

              precision    recall  f1-score   support

          -1     0.7447    0.8372    0.7882      1505
           1     0.8127    0.7110    0.7585      1495

    accuracy                         0.7743      3000
   macro avg     0.7787    0.7741    0.7734      3000
weighted avg     0.7786    0.7743    0.7734      3000

0.7743333333333333
[[1260  245]
 [ 432 1063]]
Best Score:  0.7674285714285715
Best Params:  {'clf__tol': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 3)}


In [38]:
# 获取最优模型
optimal_SVM = clf.best_estimator_

In [39]:
import pickle
# 保存模型
with open('svm-crossvalide2.pkl',"wb") as f:
    pickle.dump(optimal_SVM, f)

In [44]:
# 读取模型
with open('svm-crossvalide2.pkl',"rb") as f:
    clf_2 = pickle.load(f)

In [84]:
# 调用模型进行预测
pred = clf.predict(['i hate you, you are a bad man', 'today is a nice day let us go working'])
pred


array([1, 1])

这里有一个坑需要注意：
上面一个模型的预测和下面一个魔性的预测，区别在于，上面一个可以直接输入文本进行预测，但是下面一个却需要先“X = tv.transform(text)”，转化成tf-idf向量的形式再进行预测。
为什么会出现这样的差别呢？——原因在于训练过程步骤的不同：
*****************************************************************************************************************
在上面模型的训练过程中，因为多了一步超参数选择，设置了Pipeline，所以我先直接在文本上分割了训练集和测试集：
        text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SVC())])
        切割：
        x_train, x_test, y_train, y_test = train_test_split(data['clean_text'], data['label'],
                                                   test_size = 0.3, shuffle=True)
    然后输入超参数并开始训练：
        clf = GridSearchCV(text_clf, tuned_parameters, cv=10) # 输入超参数
        clf.fit(x_train, y_train) #开始训练
训练集与测试集就是直接在文本数据上切割的，这样在后面预测时候，直接调用predict函数输入文本进行预测就可以了。
*****************************************************************************************************************
但是下面的模型，因为没有超参数选择，没有设置Pipeline，所以直接先将文本直接转换成了tf-idf格式的向量：
        tv = TfidfVectorizer(ngram_range=(1,3), stop_words='english') 
        X = tv.fit_transform(data['clean_text'])
    然后对数据集进行训练集和测试集的切分，注意，这里切分的是已经变成向量的 X ！！！
        Xtrain, Xtest, ytrain, ytest = train_test_split(X, data['label'],
                                                   test_size = 0.3, shuffle=True)
这样一来，在之后的新数据预测中，就需要先将文本转换成tf-idf格式的向量，再用predict函数进行预测。

总而言之，就是需要记住Pipeline函数的功能，再预测的时候，注意是不是需要先加一步文本转换成向量。

推荐还是用下面的方法吧，上面不仅需要的运行时间长，而且从结果来看也就那样。

## 不使用超参数选择，仅仅常规自主设定参数的SVM（SVC）方法

In [76]:
#导入特征工程模块和数据集切分函数test_train_split
from sklearn.feature_extraction.text import TfidfVectorizer #用tf-idf转换词向量，也可用CountVectorizer，但效果不太好
from sklearn.model_selection import train_test_split

#导入分类效果评估函数
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [77]:
#TfidfVectorizer()函数是用TF-IDF算法将文本转换成数字向量，括号里设置必要的参数
#fit_transform()函数是正式传入文本，进行转换
tv = TfidfVectorizer(ngram_range=(1,3), stop_words='english') 
X = tv.fit_transform(data['clean_text'])

Xtrain, Xtest, ytrain, ytest = train_test_split(X, data['label'],
                                               test_size = 0.3, shuffle=True)

In [78]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(Xtrain, ytrain)
pred = svm.predict(Xtest)

print(accuracy_score(ytest, pred))
print(confusion_matrix(ytest, pred))
print(classification_report(ytest, pred))

0.721
[[1306  195]
 [ 642  857]]
              precision    recall  f1-score   support

          -1       0.67      0.87      0.76      1501
           1       0.81      0.57      0.67      1499

    accuracy                           0.72      3000
   macro avg       0.74      0.72      0.71      3000
weighted avg       0.74      0.72      0.71      3000



In [82]:
# 使用训练好的模型进行预测新的数据集
text = ['i hate you, you are a bad man', 'today is a nice day let us go working']

#上面tv = TfidfVectorizer(ngram_range=(1,3), stop_words='english') 已经设置好了向量转换的参数
#这里把新的文本直接转换成该文本的向量表示，输入模型，进行预测
X = tv.transform(text)

pred = svm.predict(X)
pred

array([-1,  1])